This workbook is intent to assess and clean the full_train dataset, remove missing and invalid trip.

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle
pd.set_option("max_colwidth",1000000)
pd.set_option('max_columns', 50)
import datetime
from geopy.distance import great_circle

In [2]:
train=pd.read_csv("full_train_parsed.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 22 columns):
id                     1458644 non-null object
vendor_id              1458644 non-null int64
pickup_datetime        1458644 non-null object
dropoff_datetime       1458644 non-null object
passenger_count        1458644 non-null int64
pickup_longitude       1458644 non-null float64
pickup_latitude        1458644 non-null float64
dropoff_longitude      1458644 non-null float64
dropoff_latitude       1458644 non-null float64
store_and_fwd_flag     1458644 non-null object
trip_duration          1458644 non-null int64
pickup_date            1458644 non-null object
date                   1458644 non-null object
maximum temperature    1458644 non-null int64
minimum temperature    1458644 non-null int64
average temperature    1458644 non-null float64
precipitation          1458644 non-null object
snow fall              1458644 non-null object
snow depth             1458644 non-null o

Calculate a great circle distance between pickup location and dropoff location. Create speed columns

In [4]:
train['great_circle_distance']=train.apply(lambda row: great_circle((row['pickup_latitude'],row['pickup_longitude']),(row['dropoff_latitude'],row['dropoff_longitude'])).meters,axis=1)
train['speed']=train['distance']/train['trip_duration']*3.6
train['speed_fast_route']=train['distance']/train['duration']*3.6
train['speed_great_circle']=train['great_circle_distance']/train['trip_duration']*3.6


Remove pickup_date since it's the same day as weather report date.

In [5]:
del train['pickup_date']

Remove a row where the fastest route information is missing. 

In [24]:
train[train['distance'].isnull()]

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,distance,duration,route_cooridnates,great_circle_distance,speed,speed_fast_route,speed_great_circle
482448,id2767091,1,2016-04-29 02:46:41,2016-04-29 02:48:38,1,-73.993896,40.751396,-73.993866,40.751396,N,117,2016-04-29,58,45,51.5,0.05,0.0,0,NaN,NaN,NaN,2.571391,NaN,NaN,0.07912


In [6]:
train=train[train['distance'].notnull()]
print(train.shape)

(1458643, 25)


### Define criteria to clean the data


     
     
#### 3. Trip duration must be larger than zero.


     trip_duration>0
 
 

     

In [7]:
train=train[train['trip_duration']>0]
print(train.shape)



(1458643, 25)


In [8]:
train.shape

(1458643, 25)

In [9]:
train['snow fall']=train['snow fall'].apply(lambda x: 0.001 if x=='T' else x)
train['precipitation']=train['precipitation'].apply(lambda x: 0.001 if x=='T' else x)
train['snow depth']=train['snow depth'].apply(lambda x: 0.001 if x=='T' else x)

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1457392 entries, 0 to 1458643
Data columns (total 25 columns):
id                       1457392 non-null object
vendor_id                1457392 non-null int64
pickup_datetime          1457392 non-null object
dropoff_datetime         1457392 non-null object
passenger_count          1457392 non-null int64
pickup_longitude         1457392 non-null float64
pickup_latitude          1457392 non-null float64
dropoff_longitude        1457392 non-null float64
dropoff_latitude         1457392 non-null float64
store_and_fwd_flag       1457392 non-null object
trip_duration            1457392 non-null int64
date                     1457392 non-null object
maximum temperature      1457392 non-null int64
minimum temperature      1457392 non-null int64
average temperature      1457392 non-null float64
precipitation            1457392 non-null object
snow fall                1457392 non-null object
snow depth               1457392 non-null object
dista

In [10]:
train.to_csv("train_full_parsed_clean2.1.csv",index=False)

In [11]:
train['pickup_datetime'] = pd.to_datetime(train.pickup_datetime)
train['pickup_hour'] = train['pickup_datetime'].dt.hour
train['pickup_weekday'] = train['pickup_datetime'].dt.weekday
train['pickup_month']=train['pickup_datetime'].dt.month
train['snow depth']=train['snow depth'].apply(lambda x: float(x) if x!='T' else 0.001)
train['snow']=train['snow depth'].apply(lambda x: 1 if x>0.01 else 0)

train['pickup_date']=train['pickup_datetime'].dt.date

def holiday(x):
    x1=x.date()
    if x1 in (datetime.date(2016,1,18),datetime.date(2016,2,15),datetime.date(2016,5,30),datetime.date(2016,5,28),datetime.date(2016,6,19)):
        return 1
    else:
        return 0
    
train['holiday']=train['pickup_datetime'].apply(lambda x: holiday(x))    

def hour_count(date,hour):
    return (date.date()-datetime.date(2016,1,1)).days*24+hour

train['hour_count']=train.apply(lambda row: hour_count(row['pickup_datetime'],row['pickup_hour']),axis=1)

train['great_circle_distance']=train.apply(lambda row: great_circle((row['pickup_latitude'],row['pickup_longitude']),(row['dropoff_latitude'],row['dropoff_longitude'])).meters,axis=1)

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1458643 entries, 0 to 1458643
Data columns (total 32 columns):
id                       1458643 non-null object
vendor_id                1458643 non-null int64
pickup_datetime          1458643 non-null datetime64[ns]
dropoff_datetime         1458643 non-null object
passenger_count          1458643 non-null int64
pickup_longitude         1458643 non-null float64
pickup_latitude          1458643 non-null float64
dropoff_longitude        1458643 non-null float64
dropoff_latitude         1458643 non-null float64
store_and_fwd_flag       1458643 non-null object
trip_duration            1458643 non-null int64
date                     1458643 non-null object
maximum temperature      1458643 non-null int64
minimum temperature      1458643 non-null int64
average temperature      1458643 non-null float64
precipitation            1458643 non-null object
snow fall                1458643 non-null object
snow depth               1458643 non-null floa

In [13]:
feature=['id','trip_duration','vendor_id','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude',
        'great_circle_distance','duration','speed','speed_great_circle','hour_count','passenger_count','pickup_hour','pickup_weekday','pickup_month','snow','holiday',
        'distance','route_cooridnates']
train[feature].to_csv("train_full_parsed_clean3.1.csv",index=False)

In [2]:
train=pd.read_csv("train_full_parsed_clean3.1.csv")

In [3]:
def transformation(train,istrain):
    
    train['vendor_id'] = train['vendor_id'].map( {1: 0, 2: 1} ).astype(int)
    
    temp = pd.get_dummies(train['pickup_hour'],prefix='pickup_hour')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["pickup_hour"], axis=1)

    temp = pd.get_dummies(train['pickup_weekday'],prefix='pickup_weekday')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["pickup_weekday"], axis=1)

    temp = pd.get_dummies(train['pickup_month'],prefix='pickup_month')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["pickup_month"], axis=1)

    temp = pd.get_dummies(train['passenger_count'],prefix='passenger_count')
    train = pd.concat([train,temp],axis=1)
    train = train.drop(["passenger_count"], axis=1)
    
    train['log_duration']=np.log(train['duration'].values+1)
    train=train.drop(["duration"], axis=1)
    if istrain=="Y":
        train['log_trip_duration']=np.log(train['trip_duration'].values+1)
        train=train.drop(["trip_duration"], axis=1)
    
    return train

In [4]:
train=transformation(train,istrain="Y")

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1458643 entries, 0 to 1458643
Data columns (total 75 columns):
id                       1458643 non-null object
vendor_id                1458643 non-null int32
pickup_datetime          1458643 non-null datetime64[ns]
dropoff_datetime         1458643 non-null object
pickup_longitude         1458643 non-null float64
pickup_latitude          1458643 non-null float64
dropoff_longitude        1458643 non-null float64
dropoff_latitude         1458643 non-null float64
store_and_fwd_flag       1458643 non-null object
date                     1458643 non-null object
maximum temperature      1458643 non-null int64
minimum temperature      1458643 non-null int64
average temperature      1458643 non-null float64
precipitation            1458643 non-null object
snow fall                1458643 non-null object
snow depth               1458643 non-null float64
distance                 1458643 non-null float64
route_cooridnates        1458643 non-null 

In [5]:
feature_out=['id','log_duration', 'log_trip_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance','speed_great_circle','hour_count',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7',
       'route_cooridnates']
train.to_csv("train_full_parsed_clean5.1.csv",index=False, columns=feature_out)